<a href="https://colab.research.google.com/github/anujsahani01/VoiceCloning-coqi-TTS/blob/main/Transcription_and_Data_Preperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

important installations

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
! pip install datasets

Using Medium sized wisper model to convert our audio data to text

In [4]:
a = !whisper "/content/drive/MyDrive/dataset/orignal/data-01.wav" --model medium --language en --device 'cuda' --path '/content/drive/MyDrive/dataset/text'

Saving the transcription for each audio data into a dataset.txt file

The format of the dataset.txt is

``` audio-01|transcription```

In [3]:
import whisper
import os

with open(f"/content/drive/MyDrive/dataset/dataset.txt", "w") as f:
  for dirs in os.listdir('/content/drive/MyDrive/dataset/orignal'):
    fileName = f'/content/drive/MyDrive/dataset/orignal/{dirs}'
    # Load audio
    audio = whisper.load_audio(fileName)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to('cuda')
    # Output the recognized text
    options = whisper.DecodingOptions(language=lang, without_timestamps=True)
    result = whisper.decode(model, mel, options)
    # Write into a text file
    f.write(f'{dirs[:-4]}|{result.text}\n')

Converting the .txt file to metadata.csv

In [18]:
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset


df = pd.DataFrame()

lines = []
with open('/content/drive/MyDrive/dataset/dataset.txt', 'r', encoding = 'utf-8') as f:
  text  = ' '.join(f)
  lines = text.split('\n')

In [19]:
df['data'] = lines

In [20]:
df.head(3)

,data
0,data-07|had to.
1,"data-03|As I have seen and lived more life, I..."
2,data-06|girls that take charge of their own l...


In [21]:
datasets = Dataset.from_pandas(df)
datasets

Dataset({
    features: ['data'],
    num_rows: 213
})

In [25]:
final = []
def process(data):
  if len(data['data']) == 2:
    audio_path = data['data'].split('|')[0]
    dialogue = data['data'].split('|')[1]
    speaker = 'priyanka'

    final = f"{audio_path}|{speaker}|{dialogue}"
    return {
        'final_data' : final
    }

In [26]:
final_dataset = datasets.map(process, batched = False, remove_columns = ['data'])
final_dataset

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

Dataset({
    features: ['data'],
    num_rows: 213
})

In [29]:
df_final = pd.DataFrame()
df_final['inp'] = final_dataset['data']
df_final.to_csv('/content/drive/MyDrive/dataset/metadata.csv', index = False)
df_final.to_csv('metadata.csv', index = False)